In [ ]:
# %pip install mimesis
# %pip install polars

In [1]:
import time 
from mimesis import Person, Address, Numeric
import polars as pl
from concurrent.futures import ThreadPoolExecutor



In [7]:
#write out datasets
def generate_datasets(batch_nbr: int) -> None:
    peep = Person()
    adrs = Address()
    num = Numeric()

    data = []
    for _ in range(1_000_000):
        data.append({'first_name':peep.first_name(), 'last_name':peep.last_name()
                    , 'address_txt':adrs.address(), 'zip_cd':adrs.zip_code()
                    , 'city':adrs.city(), 'state':adrs.state()
                    , 'net_worth': num.integer_number(0,50000)})
        
    pl.DataFrame(data).write_csv(f'~/test_dummy_data/polars/data{batch_nbr}.csv')
    
    return

In [8]:
def generate_datasets_parallel(num_batches: int, max_workers: int) -> None:
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(generate_datasets, range(num_batches))

In [9]:
start_time = time.time() 
generate_datasets_parallel(10, 3)
end_time = time.time()  # Record the end time
total_time = end_time - start_time  # Calculate the total time taken
print(f"Total time taken: {total_time} seconds")

In [13]:

df = pl.read_csv('~/test_dummy_data/polars/data*.csv')


## TODO: Aggregate data up, add timestamp column, write to parquet


10000000


In [31]:
df2 = df.group_by("zip_cd").agg(
     pl.col("net_worth").sum().name.suffix("_tot")
    ,pl.col("first_name").n_unique().name.suffix("_cnt")
    ,pl.col("city").count().alias("city_cnt")
)

df2.head(5)
df2.write_csv(file="~/test.csv",include_header=True, separator="|")